In [1]:
! pip install openpyxl gensim fasttext lightgbm multipledispatch razdel

     |████████████████████████████████| 242 kB 783 kB/s eta 0:00:01


In [2]:
import pandas as pd
import numpy as np
import fasttext
import razdel

import fasttext
from gensim.models import FastText

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, average_precision_score
from sklearn.model_selection import StratifiedShuffleSplit
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
import joblib

from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

In [3]:
columns = ['Общее наименование продукции', 'Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции)', 'Подкатегория продукции']
big = pd.read_excel('/kaggle/input/hackaton/big.xlsx', sheet_name='все ДС с кодами')[columns]
small = pd.read_excel('/kaggle/input/hackaton/small.xlsx', sheet_name='Данные для сопоставления')[columns]
big = big.rename(columns={"Общее наименование продукции": "product_name", 
                   "Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции)": "level",
                   "Подкатегория продукции": "category"})

small = small.rename(columns={"Общее наименование продукции": "product_name", 
                   "Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции)": "level",
                   "Подкатегория продукции": "category"})

all_data = pd.concat([big, small])

In [4]:
all_data['level_2'] = all_data['level'].str.split(';')
all_data['category'] = all_data['category'].str.split(';')

In [5]:
all_data = all_data.set_index(['product_name', 'level']).explode(['level_2', 'category']).reset_index()
all_data['level_1'] = all_data['level_2'].apply(lambda x: x.split('.')[0]+'.0').str.strip()
all_data['level_2'] = all_data['level_2'].str.strip()
all_data['category'] = all_data['category'].str.strip()

In [6]:
all_data.shape

(282165, 5)

In [7]:
dictionary = all_data[['category', 'level_1', 'level_2']].drop_duplicates()

In [8]:
dictionary.shape

(404, 3)

In [9]:
dictionary.to_csv('dictionary_level_2.csv', sep=';', index=None)

In [10]:
all_data.shape

(282165, 5)

## Предобработка

In [11]:
import re
russian_stopwords = open('/kaggle/input/hackaton/stopwords-ru.txt', 'r').read().split('\n')
okpd = pd.read_excel('/kaggle/input/hackaton/okpd.xlsx').drop_duplicates(subset=['okpd'])
okpd = okpd[okpd['okpd'].str.len()>10].reset_index(drop=True).reset_index()
dictionary = pd.read_csv('/kaggle/input/hackaton/levels.csv', sep=';', encoding='cp1251').dropna(subset=['level_2'])
dictionary['level'] = dictionary['level_1'].astype(str) + '.' + dictionary['level_2'].astype(int).astype(str)
dictionary = dictionary.drop_duplicates(subset='level')

def delete_stopwords(s):
    return ' '.join([word for word in (re.sub(r'[()\s+]', u' ', s)).split() if word.lower() not in russian_stopwords]).split()

def delete_punctuation(s):
    return re.sub(r'[®?"\'-_/.:?!1234567890()%<>;,+#$&№\s+]', u' ', s)

def get_okpd(line) -> int :
    okpd_re = re.compile('окпд2\x20*(\d{2}\.\d{2}\.\d{2}\.\d{3})')
    res = re.findall(okpd_re, line.lower())
    return res[0] if len(res) > 0 else None 

In [12]:
all_data['okpd'] = all_data['product_name'].apply(get_okpd)
all_data = all_data.join(okpd.set_index('okpd'), on = 'okpd')
all_data['clean_product_name'] = all_data['product_name'].apply(lambda x: ' '.join(delete_stopwords(delete_punctuation((x)))))

In [13]:
all_data.columns

Index(['product_name', 'level', 'category', 'level_2', 'level_1', 'okpd',
       'index', 'name', 'clean_product_name'],
      dtype='object')

## Определение категории статистикой

In [14]:
all_categories = all_data.groupby(['level_2', 'category'])['level'].count().reset_index()
all_categories = all_categories.rename(columns={'level':'count'})

In [28]:
all_categories.to_csv('all_categories.csv', sep=';')

In [15]:
def get_category_sim(product_name:str, category: str, all_categories):
    all_categories['sim'] = get_similarity(product_name, all_categories)
    all_categories = all_categories.sort_values(by=['sim', 'count'], ascending=False)
    label, probability = all_categories[['level_2', 'sim']].values[0]
    return label, probability, category==label

def get_similarity(product_name:str, all_categories):
    probability = []
    for category in all_categories:
        probability.append(fuzz.token_sort_ratio(short_rp_name, category)/100)
    return probability#all_categories['category'].apply(lambda x: ).values.tolist()

In [ ]:
# all_categories_list = all_categories[all_categories['count']<50]['category'].tolist()
# print(len(all_categories_list))
# all_data['features'] = all_data['clean_product_name'].progress_apply(lambda x: get_similarity(x, all_categories_list ))

## Генерация фичей

In [26]:
from scipy.spatial.distance import cosine
from nltk.util import ngrams
from fuzzywuzzy import fuzz


def texts_distance(text1, text2, model):
    try:
        A = get_vec(text1, model)
        B = get_vec(text2, model)
        return cosine(A, B)
    except:
        return 1


def get_vec(text, model):
    if len(text) != 0:
        try:
            return model.get_sentence_vector(text)
        except:
            return np.array([[0]*dim_ft])
    else:
        return np.array([[0]*dim_ft])
    
    
def get_features(data, column1, column2):
    #считаем пересечение между сетами сущностей двух названий
    data['intersection'] = data.progress_apply(lambda x: len(set(x[column1+'_tokens']) & set(x[column2+'_tokens'])), axis = 1)
    #считаем объединение сетов сущностей двух названий
    data['union'] = data.progress_apply(lambda x: len(set(x[column1+'_tokens']) | set(x[column2+'_tokens'])), axis = 1)
    #считаем коэф. жаккара пересечение/объединение
    data['jaccard'] = data.progress_apply(lambda x: x['intersection'] / x['union'] if x['union'] != 0 else 0, axis = 1)
    
    #считаем пересечение между сетами сущностей двух названий
    data['intersection_ngrams'] = data.progress_apply(lambda x: len(set(x[column1+'_bigrams']) & set(x[column1+'_bigrams'])), axis = 1)
    #считаем объединение сетов сущностей двух названий
    data['union_ngrams'] = data.progress_apply(lambda x: len(set(x[column1+'_bigrams']) | set(x[column2+'_bigrams'])), axis = 1)
    #считаем коэф. жаккара пересечение/объединение
    data['jaccard_ngrams'] = data.progress_apply(lambda x: x['intersection_ngrams'] / x['union_ngrams'] if x['union_ngrams'] !=0 else 0, axis = 1)    
    
                                               
    #считаем пересечение первых двух слов
    data['intersection_10w'] = data.progress_apply(lambda x: len(set(x[column1+'_first_10w']) & set(x[column2+'_first_10w'])), axis = 1)
    #считаем пересечение первых двух слов
    data['intersection_5w'] = data.progress_apply(lambda x: len(set(x[column1+'_first_5w']) & set(x[column2+'_first_5w'])), axis = 1)
    #считаем пересечение первых двух слов
    data['intersection_2w'] = data.progress_apply(lambda x: len(set(x[column1+'_first_2w']) & set(x[column2+'_first_2w'])), axis = 1)
    #считаем пересечение первых слов
    data['intersection_1w'] = data.progress_apply(lambda x: len(set(x[column1+'_first_1w']) & set(x[column2+'_first_1w'])), axis = 1)
    #считаем метрику похожести
    data['fuzzy_similarity_partial_left'] = data.fillna('').progress_apply(lambda x: fuzz.partial_ratio(x[column1], x[column2]), axis = 1)
    data['fuzzy_similarity'] = data.fillna('').progress_apply(lambda x: fuzz.ratio(x[column1], x[column2]), axis = 1)
    data['fuzzy_similarity_tokens'] = data.fillna('').progress_apply(lambda x: fuzz.token_set_ratio(x[column1], x[column2]), axis = 1)

    return data
                                                                                         
def get_ngrams(text, n = 2):
    if text == '':
        return []
    else:
        return [''.join(_) for _ in ngrams(text, n = n)]
                                                                                         
def tokenize(corpus):
    return [[word.text for word in razdel.tokenize(text.lower())] 
                       for text in corpus]

def get_first(df, n = 1):
    return df.progress_apply(lambda x: x[:n])

In [30]:
all_data['product_name_tokens'] = tokenize(tqdm(all_data.product_name))
all_data['category_tokens'] = tokenize(tqdm(all_data.category.fillna('')))

100%|██████████| 282165/282165 [01:04<00:00, 4371.98it/s] 


In [33]:
all_data['product_name_first_5w'] = get_first(all_data['product_name_tokens'], 5)
all_data['category_first_5w'] = get_first(all_data['category_tokens'], 5)
all_data['intersection_5w'] = all_data.progress_apply(lambda x: len(set(x['product_name_first_5w']) & set(x['category_first_5w'])), axis = 1)

100%|██████████| 282165/282165 [00:07<00:00, 39278.49it/s]


In [ ]:
all_data['product_name_bigrams'] = all_data['product_name'].progress_apply(lambda x: get_ngrams(x, 2))
all_data['category_1_bigrams'] = all_data['category_1'].fillna('').progress_apply(lambda x: get_ngrams(x, 2))

In [ ]:
all_data['product_name_first_10w'] = get_first(all_data['product_name_tokens'], 10)
all_data['category_1_first_10w'] = get_first(all_data['category_1_tokens'], 10)
all_data['product_name_first_5w'] = get_first(all_data['product_name_tokens'], 5)
all_data['category_1_first_5w'] = get_first(all_data['category_1_tokens'], 5)
all_data['product_name_first_2w'] = get_first(all_data['product_name_tokens'], 2)
all_data['category_1_first_2w'] = get_first(all_data['category_1_tokens'], 2)
all_data['product_name_first_1w'] = get_first(all_data['product_name_tokens'], 1)
all_data['category_1_first_1w'] = get_first(all_data['category_1_tokens'], 1)

In [ ]:
all_data = get_features(all_data, 'product_name', 'category_1')

In [ ]:
all_data[['product_name', 'level', 'okpd', 'level_1', 'category_1', 'level_2',
       'category_2', 'name', 'clean_product_name', 
       'intersection', 'union', 'jaccard', 'intersection_ngrams',
       'union_ngrams', 'jaccard_ngrams', 'intersection_10w', 'intersection_5w',
       'intersection_2w', 'intersection_1w', 'fuzzy_similarity_partial_left',
       'fuzzy_similarity', 'fuzzy_similarity_tokens']].to_csv('dataset_with_features.csv', sep=';')

## Тестирование модели

## FastText обучение

In [17]:
open('all_text.txt', 'w').write('\n'.join(all_data['clean_product_name'].str.lower().str.strip().tolist()))

34400618

In [18]:
%%time
fb_model = fasttext.train_unsupervised('/kaggle/working/all_text.txt',
                                       dim=254, ws=5, minCount=5, neg=25, 
                                       minn=3, maxn=5, wordNgrams=5, lr=0.15, epoch=30, loss='hs',
                                       model='skipgram', verbose=3)

Read 4M words
Number of words:  21562
Number of labels: 0
Progress: 100.0% words/sec/thread:   36064 lr:  0.000015 avg.loss:  3.057750 ETA:   0h 0m 0s  0.3% words/sec/thread:   53227 lr:  0.149484 avg.loss:  5.172256 ETA:   0h14m37s 0.147068 avg.loss:  5.890268 ETA:   0h17m26s 0.143022 avg.loss:  6.314101 ETA:   0h20m 0s  4.8% words/sec/thread:   36743 lr:  0.142735 avg.loss:  6.333260 ETA:   0h20m13s  5.2% words/sec/thread:   34898 lr:  0.142205 avg.loss:  6.401134 ETA:   0h21m12s  6.2% words/sec/thread:   34118 lr:  0.140759 avg.loss:  6.487196 ETA:   0h21m28s  6.4% words/sec/thread:   33875 lr:  0.140423 avg.loss:  6.503178 ETA:   0h21m34s  7.9% words/sec/thread:   34499 lr:  0.138089 avg.loss:  6.596118 ETA:   0h20m50s 11.5% words/sec/thread:   36606 lr:  0.132711 avg.loss:  6.597586 ETA:   0h18m52s 13.4% words/sec/thread:   36550 lr:  0.129862 avg.loss:  6.542646 ETA:   0h18m29s 15.0% words/sec/thread:   35509 lr:  0.127563 avg.loss:  6.508433 ETA:   0h18m42s 15.5% words/sec/threa

CPU times: user 1h 4min 52s, sys: 7.35 s, total: 1h 5min
Wall time: 21min 42s


Progress: 100.0% words/sec/thread:   36065 lr:  0.000000 avg.loss:  3.057495 ETA:   0h 0m 0s


In [19]:
fb_model.save_model('fb_model.bin') 

## Предсказание модели  

In [35]:
from sklearn.model_selection import train_test_split
import numpy as np
from lightgbm import LGBMClassifier
import joblib

## Train level_1 
down sampling

In [40]:
all_data.columns

Index(['product_name', 'level', 'category', 'level_2', 'level_1', 'okpd',
       'index', 'name', 'clean_product_name', 'product_name_tokens',
       'product_name_first_5w', 'category_tokens', 'category_first_5w',
       'intersection_5w'],
      dtype='object')

In [51]:
cnts = all_data['level_1'].value_counts()
normal_categories = cnts[cnts>10].index.tolist()

all_data = all_data[all_data['level_1'].isin(normal_categories)]
all_data = all_data.dropna(subset=['level_1'])

In [52]:
all_data_train = pd.concat([all_data[[ 'level_2', 'level_1', 
       'clean_product_name','intersection_5w']], pd.get_dummies(all_data['name'])], axis=1)

In [53]:
all_data_train.shape

(251075, 93)

In [57]:
all_data_filtred = pd.concat([ all_data_train[all_data_train.level_1=='9300'].drop_duplicates().sample(n=12000, replace=True, random_state=42),
                                all_data_train[all_data_train.level_1!='9300']])

In [61]:
train, test = train_test_split(all_data_filtred[['clean_product_name', 'level_1']], test_size=0.3, random_state=42)
train, val = train_test_split(all_data_filtred[['clean_product_name', 'level_1']], test_size=0.15, random_state=42)

In [59]:
ft_model = fasttext.load_model('/kaggle/working/fb_model.bin')

In [62]:
%%time
train_vectors = np.array([ft_model.get_sentence_vector(text) for text in train['clean_product_name'].str.replace('\n', ' ').str.lower()])
val_vectors = np.array([ft_model.get_sentence_vector(text) for text in val['clean_product_name'].str.replace('\n', ' ').str.lower()])
test_vectors = np.array([ft_model.get_sentence_vector(text) for text in test['clean_product_name'].str.replace('\n', ' ').str.lower()])

CPU times: user 12.5 s, sys: 149 ms, total: 12.7 s
Wall time: 12.6 s


In [63]:
lgbm_params = {
    'n_estimators': 300,
    'max_depth': 10,
    'learning_rate': 0.05,
    'n_jobs': 7,
    'random_state':42,
    'first_metric_only':True,
    'is_unbalance':True
}

model = LGBMClassifier(**lgbm_params)
model.fit(train_vectors, train['level_1'], verbose=True, eval_set=(val_vectors, val['level_1']),  early_stopping_rounds = 15)

[1]	valid_0's multi_logloss: 2.15868
Training until validation scores don't improve for 15 rounds
[2]	valid_0's multi_logloss: 1.91887
[3]	valid_0's multi_logloss: 1.73153
[4]	valid_0's multi_logloss: 1.57749
[5]	valid_0's multi_logloss: 1.45749
[6]	valid_0's multi_logloss: 1.36261
[7]	valid_0's multi_logloss: 1.26484
[8]	valid_0's multi_logloss: 1.18579
[9]	valid_0's multi_logloss: 1.1308
[10]	valid_0's multi_logloss: 1.06305
[11]	valid_0's multi_logloss: 1.00711
[12]	valid_0's multi_logloss: 0.956382
[13]	valid_0's multi_logloss: 0.911294
[14]	valid_0's multi_logloss: 0.871812
[15]	valid_0's multi_logloss: 0.834394
[16]	valid_0's multi_logloss: 0.799779
[17]	valid_0's multi_logloss: 0.76825
[18]	valid_0's multi_logloss: 0.739836
[19]	valid_0's multi_logloss: 0.713536
[20]	valid_0's multi_logloss: 0.6896
[21]	valid_0's multi_logloss: 0.667425
[22]	valid_0's multi_logloss: 0.646894
[23]	valid_0's multi_logloss: 0.628224
[24]	valid_0's multi_logloss: 0.610613
[25]	valid_0's multi_loglos

LGBMClassifier(first_metric_only=True, is_unbalance=True, learning_rate=0.05,
               max_depth=10, n_estimators=300, n_jobs=7, random_state=42)

In [64]:
from sklearn import metrics

y_predicted = model.predict(test_vectors)
print(metrics.classification_report(test['level_1'], y_predicted))

              precision    recall  f1-score   support

        1482       0.94      0.89      0.92      1700
        1483       0.84      0.90      0.87       947
        2180       1.00      0.98      0.99       421
        2221       0.98      0.98      0.98       326
        2293       0.96      0.98      0.97      4728
        2321       0.26      0.36      0.30        14
        2364       0.99      1.00      0.99      3738
        2380       0.94      0.96      0.95       423
        2381       0.98      0.97      0.97       998
        2386       0.79      0.38      0.51       178
        2388       1.00      0.96      0.98       505
        2514       0.90      0.91      0.91        93
        3412       0.94      0.94      0.94       269
        3414       0.97      0.96      0.96       783
        3481       0.97      0.89      0.93       332
        3482       0.98      0.99      0.98      1503
        3530       0.94      0.96      0.95        83
        5530       0.98    

In [65]:
joblib.dump(model, 'level_1.pkl')

['level_1.pkl']

## Level1 + features 

In [69]:
columns = [ 
       'Анальгетики', 'Ангиопротекторы', 'Анестетики', 'Антикоагулянты',
       'Антисептики и дезинфицирующие препараты',
       'Антисептики и дезинфицирующие препараты прочие',
       'Антисептики и противомикробные препараты для лечения гинекологических заболеваний',
       'Бета-адреноблокаторы', 'Бетон, готовый для заливки (товарный бетон)',
       'Блокаторы кальциевых каналов', 'Вазодилататоры периферические',
       'Вещества контрастные', 'Гемостатики',
       'Глюкокортикостероиды для местного лечения заболеваний кожи',
       'Гормоны гипоталамуса и гипофиза и их аналоги', 'Гормоны половые',
       'Дерматопротекторы', 'Диуретики', 'Добавки минеральные',
       'Иммунодепрессанты', 'Иммуномодуляторы', 'Миорелаксанты',
       'Препараты антианемические',
       'Препараты антибактериальные для системного использования',
       'Препараты антибактериальные и противомикробные для лечения заболеваний кожи',
       'Препараты антигистаминные системного действия',
       'Препараты гиполипидемические', 'Препараты гипотензивные',
       'Препараты гормональные для системного использования, кроме половых гормонов',
       'Препараты для лечения гинекологических заболеваний прочие',
       'Препараты для лечения заболеваний глаз',
       'Препараты для лечения заболеваний горла',
       'Препараты для лечения заболеваний кожи прочие',
       'Препараты для лечения заболеваний костей',
       'Препараты для лечения заболеваний нервной системы прочие',
       'Препараты для лечения заболеваний опорно-двигательного аппарата другие',
       'Препараты для лечения заболеваний органов дыхания прочие',
       'Препараты для лечения заболеваний печени и желчевыводящих путей',
       'Препараты для лечения заболеваний пищеварительного тракта и обмена веществ',
       'Препараты для лечения заболеваний сердца',
       'Препараты для лечения заболеваний уха',
       'Препараты для лечения заболеваний щитовидной железы',
       'Препараты для лечения заболеваний, связанных с нарушением кислотности',
       'Препараты для лечения зуда кожи, включая антигистаминные препараты и анестетики',
       'Препараты для лечения нервной системы',
       'Препараты для лечения обструктивных заболеваний дыхательных путей',
       'Препараты для лечения ожирения (исключая диетические продукты)',
       'Препараты для лечения органов дыхательной системы',
       'Препараты для лечения ран и язв',
       'Препараты для лечения сахарного диабета',
       'Препараты для лечения сердечно-сосудистой системы',
       'Препараты для лечения угревой сыпи',
       'Препараты для лечения урологических заболеваний',
       'Препараты для лечения функциональных расстройств желудочно-кишечного тракта',
       'Препараты для наружного применения при болевом синдроме при заболеваниях костно-мышечной системы',
       'Препараты для уничтожения эктопаразитов (включая чесоточного клеща), инсектициды и репелленты',
       'Препараты назальные', 'Препараты общетонизирующие',
       'Препараты противовирусные для системного применения',
       'Препараты противовоспалительные и противоревматические',
       'Препараты противогельминтные',
       'Препараты противогрибковые для лечения заболеваний кожи',
       'Препараты противогрибковые для системного использования',
       'Препараты противодиарейные, кишечные противовоспалительные и противомикробные',
       'Препараты противомикробные для системного использования',
       'Препараты противоопухолевые',
       'Препараты противоопухолевые гормональные',
       'Препараты противопаркинсонические', 'Препараты противопротозойные',
       'Препараты противоэпилептические', 'Препараты психотропные',
       'Препараты слабительные', 'Препараты стоматологические',
       'Препараты фармацевтические прочие',
       'Препараты, активные в отношении микобактерий',
       'Препараты, влияющие на кроветворение и кровь',
       'Препараты, влияющие на систему ренин-ангиотензин',
       'Препараты, применяемые при кашле и простудных заболеваниях',
       'Препараты, регулирующие обмен кальция',
       'Препараты, способствующие пищеварению, включая ферментные препараты',
       'Психоаналептики', 'Растворы плазмозамещающие и перфузионные',
       'Растворы строительные', 'Смеси строительные',
       'Средства нелечебные прочие', 'Средства питания',
       'Тетрациклины и их производные',
       'Устройства для коммутации или защиты электрических цепей на напряжение более 1 кВ прочие, не включенные в другие группировки',
       'Экстракты желез и прочих органов человеческого или животного происхождения', 'clean_product_name', 'level_1']

In [71]:
train, test = train_test_split(all_data_filtred[columns], test_size=0.3, random_state=42)
train, val = train_test_split(all_data_filtred[columns], test_size=0.15, random_state=42)

In [ ]:
y_predicted = model.predict(test_vectors)
print(metrics.classification_report(test['level_1'], y_predicted))

In [72]:
%%time
train_vectors = np.array([ft_model.get_sentence_vector(text) for text in train['clean_product_name'].str.replace('\n', ' ').str.lower()])
val_vectors = np.array([ft_model.get_sentence_vector(text) for text in val['clean_product_name'].str.replace('\n', ' ').str.lower()])
test_vectors = np.array([ft_model.get_sentence_vector(text) for text in test['clean_product_name'].str.replace('\n', ' ').str.lower()])

CPU times: user 12.5 s, sys: 113 ms, total: 12.6 s
Wall time: 12.5 s


In [77]:
train_data = np.hstack([train_vectors, train[columns[:-2]]])
val_data = np.hstack([val_vectors, val[columns[:-2]]])
test_data = np.hstack([test_vectors, test[columns[:-2]]])

In [79]:
lgbm_params = {
    'n_estimators': 300,
    'max_depth': 10,
    'learning_rate': 0.05,
    'n_jobs': 7,
    'random_state':42,
    'first_metric_only':True,
    'is_unbalance':True
}

model = LGBMClassifier(**lgbm_params)
model.fit(train_data, train['level_1'], verbose=True, eval_set=(val_data, val['level_1']),  early_stopping_rounds = 5)

[1]	valid_0's multi_logloss: 2.15868
Training until validation scores don't improve for 5 rounds
[2]	valid_0's multi_logloss: 1.91887
[3]	valid_0's multi_logloss: 1.73153
[4]	valid_0's multi_logloss: 1.57749
[5]	valid_0's multi_logloss: 1.45749
[6]	valid_0's multi_logloss: 1.36261
[7]	valid_0's multi_logloss: 1.26484
[8]	valid_0's multi_logloss: 1.18579
[9]	valid_0's multi_logloss: 1.1308
[10]	valid_0's multi_logloss: 1.06305
[11]	valid_0's multi_logloss: 1.00711
[12]	valid_0's multi_logloss: 0.956382
[13]	valid_0's multi_logloss: 0.911294
[14]	valid_0's multi_logloss: 0.871812
[15]	valid_0's multi_logloss: 0.834394
[16]	valid_0's multi_logloss: 0.799779
[17]	valid_0's multi_logloss: 0.76825
[18]	valid_0's multi_logloss: 0.739836
[19]	valid_0's multi_logloss: 0.713536
[20]	valid_0's multi_logloss: 0.6896
[21]	valid_0's multi_logloss: 0.667425
[22]	valid_0's multi_logloss: 0.646894
[23]	valid_0's multi_logloss: 0.628224
[24]	valid_0's multi_logloss: 0.610613
[25]	valid_0's multi_logloss

LGBMClassifier(first_metric_only=True, is_unbalance=True, learning_rate=0.05,
               max_depth=10, n_estimators=300, n_jobs=7, random_state=42)

In [80]:
from sklearn import metrics

y_predicted = model.predict(test_data)
print(metrics.classification_report(test['level_1'], y_predicted))

              precision    recall  f1-score   support

        1482       0.94      0.89      0.92      1700
        1483       0.84      0.90      0.87       947
        2180       1.00      0.98      0.99       421
        2221       0.98      0.98      0.98       326
        2293       0.96      0.98      0.97      4728
        2321       0.26      0.36      0.30        14
        2364       0.99      1.00      0.99      3738
        2380       0.94      0.96      0.95       423
        2381       0.98      0.97      0.97       998
        2386       0.79      0.38      0.51       178
        2388       1.00      0.96      0.98       505
        2514       0.90      0.91      0.91        93
        3412       0.94      0.94      0.94       269
        3414       0.97      0.96      0.96       783
        3481       0.97      0.89      0.93       332
        3482       0.98      0.99      0.98      1503
        3530       0.94      0.96      0.95        83
        5530       0.98    

In [81]:
joblib.dump(model, 'model_level_1_features.pkl')

['model_level_1_features.pkl']

## Level 2

In [91]:
cnts = all_data['level_2'].value_counts()
normal_categories = cnts[cnts>10].index.tolist()

all_data = all_data[all_data['level_2'].isin(normal_categories)]
all_data = all_data.dropna(subset=['level_2'])

In [92]:
all_data_train = pd.concat([all_data[[ 'level_2',
       'clean_product_name']], pd.get_dummies(all_data['name'])], axis=1)

In [105]:
all_data_filtred = pd.concat([ all_data_train[all_data_train.level_2.isin(['9300.1', '2364.1', '5990.1'])].drop_duplicates().sample(n=10000, replace=True, random_state=42),
                                all_data_train[~all_data_train.level_2.isin(['9300.1', '2364.1', '5990.1'])]])

In [107]:
%%time
train_vectors = np.array([ft_model.get_sentence_vector(text) for text in train['clean_product_name'].str.replace('\n', ' ').str.lower()])
val_vectors = np.array([ft_model.get_sentence_vector(text) for text in val['clean_product_name'].str.replace('\n', ' ').str.lower()])
test_vectors = np.array([ft_model.get_sentence_vector(text) for text in test['clean_product_name'].str.replace('\n', ' ').str.lower()])

CPU times: user 12.5 s, sys: 86.6 ms, total: 12.5 s
Wall time: 12.5 s


In [108]:
lgbm_params = {
    'n_estimators': 300,
    'max_depth': 10,
    'learning_rate': 0.05,
    'n_jobs': 7,
    'random_state':42,
    'first_metric_only':True,
    'is_unbalance':True
}

model = LGBMClassifier(**lgbm_params)
model.fit(train_vectors, train['level_2'], verbose=True, eval_set=(val_vectors, val['level_2']),  early_stopping_rounds = 5)

[1]	valid_0's multi_logloss: 3.76809
Training until validation scores don't improve for 5 rounds
[2]	valid_0's multi_logloss: 5.03279
[3]	valid_0's multi_logloss: 6.08856
[4]	valid_0's multi_logloss: 6.62396
[5]	valid_0's multi_logloss: 7.7812
[6]	valid_0's multi_logloss: 8.40228
Early stopping, best iteration is:
[1]	valid_0's multi_logloss: 3.76809
Evaluated only: multi_logloss


LGBMClassifier(first_metric_only=True, is_unbalance=True, learning_rate=0.05,
               max_depth=10, n_estimators=300, n_jobs=7, random_state=42)

In [110]:
from sklearn import metrics

y_predicted = model.predict(test_vectors)
print(metrics.classification_report(test['level_2'], y_predicted))

              precision    recall  f1-score   support

        1482       0.12      0.50      0.19         6
      1482.1       0.94      0.67      0.79      1756
        1483       0.01      0.20      0.03         5
      1483.1       0.85      0.62      0.72       972
        2180       0.16      0.56      0.24        16
      2180.1       0.96      0.66      0.79       373
        2221       0.27      0.75      0.40         8
      2221.1       1.00      0.84      0.91       259
      2221.2       0.43      0.83      0.57        24
      2221.3       0.64      0.50      0.56        50
      2221.4       0.31      0.92      0.47        12
        2293       0.58      0.01      0.02       697
      2293.1       0.86      0.09      0.16      1172
      2293.2       0.91      0.25      0.39      1005
      2293.3       0.84      0.49      0.62       307
      2293.4       0.73      0.37      0.49       198
      2293.5       0.94      0.16      0.28      1093
      2293.6       0.43    

In [111]:
joblib.dump(model, 'model_level_2_features.pkl')

['model_level_2_features.pkl']

## level2 tfidf 

In [115]:
import tqdm
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def tokenize_with_razdel(text):
    tokens = [token.text for token in razdel.tokenize(text)]
    
    return tokens

def evaluate_vectorizer(vectorizer):
    train_vectors = vectorizer.fit_transform(train['clean_product_name'])
    test_vectors = vectorizer.transform(test['clean_product_name'])
    joblib.dump(vectorizer, 'vectorizer_level_2.pkl')

    
    clf = LinearSVC(random_state=42)
    
    clf.fit(train_vectors, train['level_2'])
    joblib.dump(clf, 'model_level_2.pkl')
    
    predictions = clf.predict(test_vectors)
    
    print(classification_report(test['level_2'], predictions))
    
    return predictions

In [113]:
train, test = train_test_split(all_data[['clean_product_name', 'level_2']], test_size=0.3, random_state=42)

In [116]:
evaluate_vectorizer(TfidfVectorizer(min_df=2, tokenizer=tokenize_with_razdel))

              precision    recall  f1-score   support

        1482       1.00      0.22      0.36         9
      1482.1       0.92      0.93      0.92      1712
        1483       0.00      0.00      0.00         4
      1483.1       0.85      0.90      0.87       993
        2180       0.33      0.08      0.12        13
      2180.1       0.96      0.97      0.97       391
        2221       0.00      0.00      0.00         2
      2221.1       0.98      0.97      0.98       261
      2221.2       0.84      0.80      0.82        20
      2221.3       0.82      0.94      0.88        54
      2221.4       1.00      0.82      0.90        11
        2293       0.57      0.53      0.55       693
      2293.1       0.61      0.57      0.59      1129
      2293.2       0.66      0.69      0.67       985
      2293.3       0.78      0.67      0.72       315
      2293.4       0.73      0.55      0.63       184
      2293.5       0.71      0.79      0.75      1137
      2293.6       0.49    

array(['9398.1', '1483.1', '9436.1', ..., '3481.4', '3482.2', '2381.3'],
      dtype=object)

## Предсказание для сервиса

In [151]:
import fasttext
from lightgbm import LGBMClassifier
import joblib
from multipledispatch import dispatch

class ProductNameClassifier:
    def __init__(self):
        self.ft_model = fasttext.load_model('fb_model.bin')
        self.model_level_1 = joblib.load('level_1.pkl')
        self.vectorizer = joblib.load('vectorizer_level_2.pkl')
        self.model_level_2 = joblib.load('model_level_2.pkl')
        self.all_categories = pd.read_csv('all_categories.csv', sep=';')
    
    @dispatch(str)
    def predict(self, text:str) -> (int, float):
        """
        Функция для обработки текста 
        """
        text = text.replace('\n', ' ')
        text = ' '.join(self.delete_stopwords(self.delete_punctuation((text))))
        text_vector = self.ft_model.get_sentence_vector(text).reshape(1, -1)
        label = int(self.model_level_1.predict(text_vector)[0])
        probability = self.model_level_1.predict_proba(text_vector)
        return label, probability.max()  
    
    @dispatch(str)
    def predict_level_2(self, text:str) -> (int, float):
        """
        Функция для обработки текста 
        """
        text = text.replace('\n', ' ')
        text = ' '.join(self.delete_stopwords(self.delete_punctuation((text))))
        text_vector = self.vectorizer.transform([text])
        label = self.model_level_2.predict(text_vector)[0]
        probability = self.model_level_2.predict_proba(text_vector)
        return label, probability.max() 
    
    @dispatch(str, int)
    def predict(self, text:str, user_label:int) -> (str, float, int):
        """
        Функция для обработки текста и метки 
        """
        text = text.replace('\n', ' ')
        text = ' '.join(self.delete_stopwords(self.delete_punctuation((text))))
        text_vector = self.ft_model.get_sentence_vector(text).reshape(1, -1)
        label = self.model_level_1.predict(text_vector)[0]
        probability = self.model_level_1.predict_proba(text_vector)
        is_equal = label == user_label
        return label, probability.max(), is_equal
    
    
    def get_category_sim(self, product_name:str, category: str):
        all_categories['sim'] = self.get_similarity(product_name, self.all_categories)
        all_categories = all_categories.sort_values(by=['sim', 'count'], ascending=False)
        label, probability = all_categories[['level_2', 'sim']].values[0]
        return label, probability, category==label

    def get_similarity(self, product_name:str):
        probability = []
        for category in self.all_categories:
            probability.append(fuzz.token_sort_ratio(short_rp_name, category)/100)
        return probability
    
    
    def delete_stopwords(self, s):
        return ' '.join([word for word in (re.sub(r'[()\s+]', u' ', s)).split() if word.lower() not in russian_stopwords]).split()

    
    def delete_punctuation(self, s):
        return re.sub(r'[®?"\'-_/.:?!1234567890()%<>;,+#$&№\s+]', u' ', s)
    
    def get_okpd(self, line) -> int :
        okpd_re = re.compile('окпд2\x20*(\d{2}\.\d{2}\.\d{2}\.\d{3})')
        res = re.findall(okpd_re, line.lower())
        return res[0] if len(res) > 0 else None 

In [148]:
df = pd.DataFrame.from_dict({'product_name': {0: 'Парацетамол таблетки 500 мг 10 шт., упаковки ячейковые контурные (2), пачки картонные, рег № ЛС-001364 от 06.08.2010, серия 190618, партия 59110 упаковок, годен до 01.07.2022, производства  ОАО "Фармстандарт-Лексредства", ИНН 4631002737, 305022, Курская область, Курск, ул. 2-я Агрегатная, 1А/18, Россия, код ОКПД2 21.20.10.232 ',
  1: 'Перезаряжаемая литий-ионная батарея торговой марки HUAWEI модель HB642735ECW',
  2: 'Перезаряжаемая литий-ионная батарея торговой марки vivo модель B-E8',
  3: 'Аппарат вакуумно-лазерной терапии стоматологический АВЛТ-"ДЕСНА" (по методу Кулаженко-Лепилина)',
  4: 'Блоки оконные и балконные дверные из алюминиевых профилей системы "INICIAL" серии "IW63" фирмы ООО "Урало-Сибирская профильная компания"'},
 'category_1': {0: 9300, 1: 3482, 2: 3482, 3: 9444, 4: 5270},
 'model_label': {0: 9300.0, 1: 3482.0, 2: 3482.0, 3: 9441.0, 4: 5772.0},
 'probability': {0: 0.6942050498562966,
  1: 0.9989457655246328,
  2: 0.9989539612756103,
  3: 0.703270209758987,
  4: 0.9999890387091874}})

In [ ]:
product_name_clf = ProductNameClassifier()

product_name_clf.predict_level_2('Перезаряжаемая литий-ионная батарея торговой марки vivo модель B-E8')

In [139]:
product_name_clf = ProductNameClassifier()

#если есть и метка от пользователя и текст
label, probability, is_equal = product_name_clf.predict(big['product_name'][0], 9300)
print(label, probability, is_equal )

#если только текст
label, probability = product_name_clf.predict(big['product_name'][0])
print(label, probability)

#Скоринг всего файла с наличием метки от пользователя
df[['model_label', 'probability', 'is_equal']] = None
df[['model_label', 'probability', 'is_equal']] = df[['product_name', 'category_1']].apply(lambda x: product_name_clf.predict(*x),
                                                                                                    axis=1,
                                                                                                    result_type="expand")

#Скоринг всего файла без метки от пользователя
df[['model_label', 'probability']] = None
df[['model_label', 'probability']] = df[['product_name']].apply(lambda x: product_name_clf.predict(*x),
                                                                                                    axis=1,
                                                                                                    result_type="expand")

9300 0.9767396655216843 False
9300 0.9767396655216843


In [133]:
label, probability, is_equal = product_name_clf.predict(big['product_name'][0], 9300)

,product_name,category_1,model_label,probability,is_equal
0,"Парацетамол таблетки 500 мг 10 шт., упаковки я...",9300,9300.0,0.976740,False
1,Перезаряжаемая литий-ионная батарея торговой м...,3482,3482.0,0.972501,False
2,Перезаряжаемая литий-ионная батарея торговой м...,3482,3482.0,0.972775,False
3,Аппарат вакуумно-лазерной терапии стоматологич...,9444,9444.0,0.448736,False
4,Блоки оконные и балконные дверные из алюминиев...,5270,2293.0,0.155847,False


In [ ]:
multipledispatch==0.6.0
lightgbm==3.2.1
joblib==1.0.1
fasttext==0.9.2
pandas==1.3.4
numpy==1.19.5
scipy==1.7.1
scikit-learn==0.23.2

In [ ]:
label, probability

In [ ]:
pip list |grep scipy

In [ ]:
df = pd.DataFrame.from_dict(
    {'product_name': {0: 'Парацетамол таблетки 500 мг 10 шт., упаковки ячейковые контурные (2), пачки картонные, рег № ЛС-001364 от 06.08.2010, серия 190618, партия 59110 упаковок, годен до 01.07.2022, производства  ОАО "Фармстандарт-Лексредства", ИНН 4631002737, 305022, Курская область, Курск, ул. 2-я Агрегатная, 1А/18, Россия, код ОКПД2 21.20.10.232 ',
      1: 'Перезаряжаемая литий-ионная батарея торговой марки HUAWEI модель HB642735ECW',
      2: 'Перезаряжаемая литий-ионная батарея торговой марки vivo модель B-E8',
      3: 'Аппарат вакуумно-лазерной терапии стоматологический АВЛТ-"ДЕСНА" (по методу Кулаженко-Лепилина)',
      4: 'Блоки оконные и балконные дверные из алюминиевых профилей системы "INICIAL" серии "IW63" фирмы ООО "Урало-Сибирская профильная компания"'},
     'category_1': {0: 9300, 1: 3482, 2: 3482, 3: 9444, 4: 5270}
    })

product_name_clf = ProductNameClassifier()


#Скоринг всего файла без метки от пользователя
df[['model_label', 'probability']] = None
df[['model_label', 'probability']] = df[['product_name']].apply(lambda x: product_name_clf.predict(*x),
                                                                                                    axis=1,
                                                                                                    result_type="expand")
print('3:\n', df.head())
#Скоринг всего файла с наличием метки от пользователя
df[['model_label', 'probability', 'is_equal']] = None
df[['model_label', 'probability', 'is_equal']] = df[['product_name', 'category_1']].apply(lambda x: product_name_clf.predict(*x),
                                                                                                    axis=1,
                                                                                                    result_type="expand")
print('3:\n', df.head())